# Connection à la base de données 'fcd'

In [1]:
import pandas as pd
import pymongo

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# Lecture dans la base

In [2]:
cursor = collection.find()
entries = list(cursor)

#entries[:]

df = pd.DataFrame(entries)
df.head()
print('\ndone')


done


# Entrée 1 : géo

In [4]:
lon = float(input("longitude -> "))
lat = float(input("latitude -> "))
rad = float(input("rayon (km) -> "))

print('\ndone')

longitude -> -113.85
latitude -> 45.66
rayon (km) -> 15

done


# Entrée 2 : date

In [4]:
from datetime import timedelta
from datetime import datetime

year = int(input("Annee -> "))
month = int(input("Mois -> "))
day = int(input("Jour -> "))
hour = int(input("Heure -> "))
minutes = int(input("Minutes -> "))
seconds = int(input("Secondes -> "))
date = datetime(year, month, day, hour, minutes, seconds)

print('\ndone')

Annee -> 2020
Mois -> 3
Jour -> 5
Heure -> 14
Minutes -> 14
Secondes -> 21

done


# Entrée 3 : période

In [5]:
T = int(input("Période (minutes) -> "))
delta = timedelta(minutes = float(T/2))

datemin = date - delta
datemax = date + delta

datemin = datetime.timestamp(datemin)
datemax = datetime.timestamp(datemax)

print('\ndone')

Période (minutes) -> 46

done


# QUERY 1 

In [5]:
import pprint


# QUERY 1
# Calculer la densité [nombre de points] sur une Zone géographique Z et une période T 

def density_list(lon, lat, rad, datemin, datemax, option, zone, periode):

    dico={}
    query01={}
    query02={}
    center=[-1,-1]
    
    # points dans une zone Z
    if zone:
        
        center=[lon,lat]
        
        query01 = {
            "location.geo": {
                "$nearSphere": {
                    "$geometry": {
                        "type": "Point",
                        "coordinates": [ lon, lat]
                    },
                    "$minDistance": 0,
                    "$maxDistance": rad*1000 # mètres
                }
            }
        }

    # points à une période T
    if periode :
        query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}


    # requête finale
    query = {'$and': [query01, query02]}

    if option != {} :
        query = {'$and': [query, option]}
        
    liste=list(collection.find(query).distinct("asset_id"))
    dico['neighbors_asset_id']=liste
    dico['density']=len(liste)
    
    z = {}
    z['center'] = center
    z['radius'] = rad
    dico['zone'] = z
    
    return dico


collection = client.fcd.assets
#result = density_list(lon, lat, rad, datemin, datemax, {}, True, True)
#print ("density :", result['density'])

print('done')

done


# QUERY 2

In [6]:
import pprint

# QUERY 2
# Retourner le nombre de points voisins d'un asset X dans une période T.

def time_gap(current, new):

    td = abs(current - new)
    res = td/60
    return res


def asset_density(id_asset, rad, datemin, datemax):

    # les points d'un asset X
    query01 = {"asset_id": id_asset}

    # points à une période T
    query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}

    # les points d'un asset X à une période T
    query03 = {'$and': [query01, query02]}

    tmp = list(collection.find(query03))

    lon = 0
    lat = 0
    lon_average = 0
    lat_average = 0
    
    density = 0
    res = {}
    liste = []
    num = 0
    time = 0
    
    for doc in tmp:
        
        if time_gap(time, doc['recorded_at']) > 2:
            
            time = doc['recorded_at']
            
            lon = doc['location']['geo']['coordinates'][0]
            lat = doc['location']['geo']['coordinates'][1]         
            lon_average += lon
            lat_average += lat
            num += 1
            
            d_list = density_list(lon, lat, rad, datemin, datemax, {"asset_id": {"$ne": id_asset}}, True, True)

            if d_list['density'] > 0:

                for neighbor in d_list['neighbors_asset_id']:

                    if neighbor not in liste:
                        density += 1
                        liste.append(neighbor)

    if density > 0:
        lon_average = lon_average/num
        lat_average = lat_average/num
        
        res['asset_id'] = id_asset
        res['density'] = density
        res['neighbors_asset_id']= liste
        
        z = {}
        z['center'] = [lon_average, lat_average]
        z['radius'] = rad
        res['zone'] = z

    return res
  
    
collection = client.fcd.assets
'''result = asset_density('14429000019112', rad, datemin, datemax)

if result != {}:
    print('density :', result['density'])
    print('neighbors :',result['neighbors_asset_id'])
'''
print('\ndone')


done


# QUERY 3

In [5]:
# QUERY 3
# Sur la base de tous les assets, retourner le Top – k [e.g. top 10] des zones les plus denses


def top_k_zones(k, rad, datemin, datemax):
    
    # list des id des différents asset
    ids_asset = list(collection.distinct("asset_id"))

    zones = []

    for id_ in ids_asset:

        zone = asset_density(id_, rad, datemin, datemax)

        if zone != {}:
            zones.append(zone)

    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


collection = client.fcd.assets
'''zones = top_k_zones(15, rad, datemin, datemax)

print("top", len(zones), "\n----------------------")
pprint.pprint(zones)
'''
print('\ndone')


done


# Query4

In [10]:
import math

# Query 4
# Retournez le top k des périodes/ zones les plus denses 


# in meters
def distance(lat1, lat2, lon1, lon2):
    R = 6372800  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    d = 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return d


def top_k_zones2(k, rad):
    
    # toutes les points de la base de données
    base = list(collection.find())
    
    # tri selon la longitude puis la latitude
    base = sorted(base, key=lambda point:point['location']['geo']['coordinates'])
    print('sorting done')
    
    
    zones = []
    time = 0
    lon = 0
    lat = 0
    nextP = False
    
    for point in base:
        
        if distance(lat, point['location']['geo']['coordinates'][1],
                    lon, point['location']['geo']['coordinates'][0]) >= rad*1000*2:
            
            if len(zones) > 0:
                for z in zones:
                     
                    if distance(z['zone']['center'][1], point['location']['geo']['coordinates'][1],
                                z['zone']['center'][0], point['location']['geo']['coordinates'][0]) < rad*1000*2:
                        nextP = True
                        break
            if nextP:
                nextP = False
                continue
                
            lon = point['location']['geo']['coordinates'][0]
            lat = point['location']['geo']['coordinates'][1]
            zones.append(density_list(lon, lat, rad, 0, 0,{}, True, False))
            
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


def top_k_periods(k, rad, datemin, datemax):
    
    # toutes les points de la base de données
    base = list(collection.find())
    base = sorted(base, key=lambda point:point['recorded_at'], reverse=True)
    print('sorting done\n')
    
    
    zones = []
    time = 0
    
    pprint.pprint(base)

    for point in base:
        
        if time_gap(time, point['recorded_at']) > 20:
            
            time = point['recorded_at']
            lon = point['location']['geo']['coordinates'][0]
            lat = point['location']['geo']['coordinates'][1]
            
            zones.append(density_list(lon, lat, rad, 0, 0,{}, True, False))
            
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


collection = client.fcd.assets
zones = top_k_zones2(10, rad)
pprint.pprint(zones)
print('\ndone')

sorting done
[{'density': 15,
  'neighbors_asset_id': ['14429000003116',
                         '14429000005848',
                         '14429000006838',
                         '14429000008883',
                         '14429000008909',
                         '14429000010095',
                         '14429000010277',
                         '14429000014790',
                         '14429000015714',
                         '14429000016597',
                         '14429000026877',
                         '14429000030150',
                         '14429000032073',
                         '14429000034814',
                         '14429000035001'],
  'zone': {'center': [-114.137, 51.0423], 'radius': 15.0}},
 {'density': 8,
  'neighbors_asset_id': ['14256004530491',
                         '14256004535292',
                         '14256004559268',
                         '14256004569226',
                         '14256004574044',
                         '1425600

# Query 5

In [8]:
collection = client.fcd.trips

zones = top_k_zones_Periode(3, rad, datemin, datemax,False,True)
pprint.pprint(zones)
print('\ndone')

[{'center': [-1, -1],
  'density': 1790,
  'liste_points': [{'_id': ObjectId('5f19a7984f7c351931d7caea'),
                    'asset_id': '14429000008883',
                    'location': {'geo': {'coordinates': [-114.066, 51.1532],
                                         'type': 'Point'}},
                    'recorded_at': 1583413220.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7caeb'),
                    'asset_id': '14429000035001',
                    'location': {'geo': {'coordinates': [-114.06, 50.8977],
                                         'type': 'Point'}},
                    'recorded_at': 1583413227.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7caec'),
                    'asset_id': '14429000026877',
                    'location': {'geo': {'coordinates': [-113.971, 50.941],
                                         'type': 'Point'}},
                    'recorded_at': 15834132

                    'location': {'geo': {'coordinates': [-114.062, 50.8973],
                                         'type': 'Point'}},
                    'recorded_at': 1583413371.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb20'),
                    'asset_id': '14429000008834',
                    'location': {'geo': {'coordinates': [-113.57, 53.2663],
                                         'type': 'Point'}},
                    'recorded_at': 1583413374.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb21'),
                    'asset_id': '14429000019013',
                    'location': {'geo': {'coordinates': [-113.99, 51.2706],
                                         'type': 'Point'}},
                    'recorded_at': 1583413375.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb22'),
                    'asset_id': '14429000032073',
   

                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb3d'),
                    'asset_id': '14429000032073',
                    'location': {'geo': {'coordinates': [-113.991, 50.8922],
                                         'type': 'Point'}},
                    'recorded_at': 1583413448.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb3e'),
                    'asset_id': '14429000008883',
                    'location': {'geo': {'coordinates': [-114.042, 51.1534],
                                         'type': 'Point'}},
                    'recorded_at': 1583413451.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb3f'),
                    'asset_id': '14429000035001',
                    'location': {'geo': {'coordinates': [-114.063, 50.8932],
                                         'type': 'Point'}},
                    'recorded_at': 1583413451.0,
 

                    'location': {'geo': {'coordinates': [-113.971, 50.9411],
                                         'type': 'Point'}},
                    'recorded_at': 1583413502.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb5b'),
                    'asset_id': '14429000004924',
                    'location': {'geo': {'coordinates': [-79.8038, 43.703],
                                         'type': 'Point'}},
                    'recorded_at': 1583413507.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb5c'),
                    'asset_id': '14429000008834',
                    'location': {'geo': {'coordinates': [-113.572, 53.2647],
                                         'type': 'Point'}},
                    'recorded_at': 1583413513.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb5d'),
                    'asset_id': '14429000016597',
  

                   {'_id': ObjectId('5f19a7984f7c351931d7cb81'),
                    'asset_id': '14429000006838',
                    'location': {'geo': {'coordinates': [-114.035, 50.9934],
                                         'type': 'Point'}},
                    'recorded_at': 1583413580.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb82'),
                    'asset_id': '14429000030150',
                    'location': {'geo': {'coordinates': [-114.089, 50.9248],
                                         'type': 'Point'}},
                    'recorded_at': 1583413581.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7cb83'),
                    'asset_id': '14429000014790',
                    'location': {'geo': {'coordinates': [-113.971, 50.9411],
                                         'type': 'Point'}},
                    'recorded_at': 1583413582.0,
                    'trip': 4},
 

...

                    'location': {'geo': {'coordinates': [-114.036, 50.9703],
                                         'type': 'Point'}},
                    'recorded_at': 1583416343.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d120'),
                    'asset_id': '14429000015615',
                    'location': {'geo': {'coordinates': [-113.403, 53.4956],
                                         'type': 'Point'}},
                    'recorded_at': 1583416345.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d121'),
                    'asset_id': '14429000029509',
                    'location': {'geo': {'coordinates': [-79.7817, 43.2565],
                                         'type': 'Point'}},
                    'recorded_at': 1583416347.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d122'),
                    'asset_id': '14429000010095',
 

                    'recorded_at': 1583416411.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d139'),
                    'asset_id': '14429000023551',
                    'location': {'geo': {'coordinates': [-80.9289, 43.2583],
                                         'type': 'Point'}},
                    'recorded_at': 1583416414.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d13a'),
                    'asset_id': '14429000019112',
                    'location': {'geo': {'coordinates': [-113.502, 53.4365],
                                         'type': 'Point'}},
                    'recorded_at': 1583416418.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d13b'),
                    'asset_id': '14429000015615',
                    'location': {'geo': {'coordinates': [-113.418, 53.4954],
                                         'type': 'Point'}},
 

                    'location': {'geo': {'coordinates': [-79.7165, 43.8295],
                                         'type': 'Point'}},
                    'recorded_at': 1583416574.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d174'),
                    'asset_id': '14429000015615',
                    'location': {'geo': {'coordinates': [-113.439, 53.4856],
                                         'type': 'Point'}},
                    'recorded_at': 1583416577.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d175'),
                    'asset_id': '14429000019112',
                    'location': {'geo': {'coordinates': [-113.564, 53.4287],
                                         'type': 'Point'}},
                    'recorded_at': 1583416577.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d176'),
                    'asset_id': '14429000016597',
 

                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d191'),
                    'asset_id': '14429000016597',
                    'location': {'geo': {'coordinates': [-114.281, 51.0187],
                                         'type': 'Point'}},
                    'recorded_at': 1583416641.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d192'),
                    'asset_id': '14429000008883',
                    'location': {'geo': {'coordinates': [-114.044, 51.1541],
                                         'type': 'Point'}},
                    'recorded_at': 1583416642.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d193'),
                    'asset_id': '14429000008909',
                    'location': {'geo': {'coordinates': [-114.01, 51.0104],
                                         'type': 'Point'}},
                    'recorded_at': 1583416643.0,
  

                    'location': {'geo': {'coordinates': [-79.7405, 43.8469],
                                         'type': 'Point'}},
                    'recorded_at': 1583416747.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d1c2'),
                    'asset_id': '14429000019112',
                    'location': {'geo': {'coordinates': [-113.615, 53.4599],
                                         'type': 'Point'}},
                    'recorded_at': 1583416750.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d1c3'),
                    'asset_id': '14429000008909',
                    'location': {'geo': {'coordinates': [-114.001, 51.0224],
                                         'type': 'Point'}},
                    'recorded_at': 1583416751.0,
                    'trip': 4},
                   {'_id': ObjectId('5f19a7984f7c351931d7d1c4'),
                    'asset_id': '14429000010095',
 